In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd

In [2]:
df = pd.read_csv("G:/Software/Machine learning/Datasets/Titanic/train.csv")
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
#dropping columns
train_df = df.drop(["PassengerId" , "Name" , "Ticket"] , axis=1)

In [4]:
print ("Train")
print (train_df.isnull().sum())

Train
Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64


In [5]:
##### Fill in NAs
##identify mean ages
male_mean_age = train_df[train_df["Sex"]=="male"]["Age"].mean()
female_mean_age = train_df[train_df["Sex"]=="female"]["Age"].mean()
print ("female mean age: %1.0f" %female_mean_age )
print ("male mean age: %1.0f" %male_mean_age )

female mean age: 28
male mean age: 31


In [6]:
# fill the nan values 
train_df.loc[ (train_df["Sex"]=="male") & (train_df["Age"].isnull()), "Age"] = male_mean_age
train_df.loc[ (train_df["Sex"]=="female") & (train_df["Age"].isnull()), "Age"] = female_mean_age

In [7]:
### same for fare
mean_fare = train_df["Fare"].mean()

In [8]:
train_df["Cabin"] = train_df["Cabin"].fillna("X")
train_df["Embarked"] = train_df["Embarked"].fillna("S")

In [9]:
y = train_df.Survived#response 
x = train_df.drop(['Survived'],axis=1)#predictors

X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=101)#75% training

In [10]:
# defining numeric columns
pclass_feature = tf.feature_column.numeric_column('Pclass')
parch_feature = tf.feature_column.numeric_column('Parch')
fare_feature = tf.feature_column.numeric_column('Fare')
age_feature = tf.feature_column.numeric_column('Age')

In [11]:
#defining buckets for children, teens, adults and elders.
age_bucket_feature = tf.feature_column.bucketized_column(age_feature,[12,21,60])

In [12]:
#defining a categorical column with predefined values
sex_feature = tf.feature_column.categorical_column_with_vocabulary_list(
    'Sex',['female','male']
)

In [13]:
#defining a categorical columns with dynamic values
embarked_feature =  tf.feature_column.categorical_column_with_hash_bucket(
    'Embarked', 3 
)

In [14]:
cabin_feature =  tf.feature_column.categorical_column_with_hash_bucket(
    'Cabin', 100 
)


In [15]:
feature_columns = [ pclass_feature,age_feature, age_bucket_feature, parch_feature, 
                   fare_feature, embarked_feature, cabin_feature ]

In [16]:
########## Define TF Estimator
estimator = tf.estimator.LinearClassifier(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\MAHMUD~1\\AppData\\Local\\Temp\\tmp4vicefum', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
# train input function
train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
      x = X_train,
      y = y_train,
      num_epochs = None,
      shuffle = True,
      target_column = 'target',
)

In [18]:
print(tf.__version__)

2.1.0


In [19]:
estimator.train(input_fn=train_input_fn, steps=450)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Please use `layer.add_weight` method instead.
Ins

InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMV launch failed:  m=1, n=128
	 [[node linear/linear_model/linear/linear_model/linear/linear_model/Fare/weighted_sum (defined at C:\Users\Mahmudur Limon\Anaconda3\envs\gputest\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:608) ]]
	 [[linear/linear_model/linear/linear_model/linear/linear_model/Age_bucketized/weighted_sum/GatherV2_1/_259]]
  (1) Internal: Blas GEMV launch failed:  m=1, n=128
	 [[node linear/linear_model/linear/linear_model/linear/linear_model/Fare/weighted_sum (defined at C:\Users\Mahmudur Limon\Anaconda3\envs\gputest\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:608) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'linear/linear_model/linear/linear_model/linear/linear_model/Fare/weighted_sum':
  File "C:\Users\Mahmudur Limon\Anaconda3\envs\gputest\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  Fil

In [ ]:
#test on training data
results=estimator.evaluate(tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10, num_epochs=1, shuffle=False))

In [ ]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10, num_epochs=1, shuffle=False)
results=estimator.evaluate(eval_input_func)
print(results)